Create a book recommendation algorithm using K-Nearest Neighbors.

Use the Book-Crossings dataset http://www2.informatik.uni-freiburg.de/~cziegler/BX/. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

In [ ]:
# import libraries 
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt